In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from hydra.utils import instantiate
from hydra import initialize, compose
import hydra

import wandb

from data.dataManager import DataManager
from model.modelCreator import ModelCreator
from omegaconf import OmegaConf
from scripts.run import setup_model, load_model_instance

from utils.plots import vae_plots
from utils.rbm_plots import plot_rbm_histogram, plot_rbm_params

In [2]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path="config")
config=compose(config_name="config.yaml")
wandb.init(tags = [config.data.dataset_name], project=config.wandb.project, entity=config.wandb.entity, config=OmegaConf.to_container(config, resolve=True), mode='disabled')

In [6]:
new_model = True
if new_model:
    self = setup_model(config)
    # self.model = self.model.double()  # sets all model parameters to float64
else:
    self = load_model_instance(config)
    # self.model = self.model.double()


[19:40:49.168] INFO   data.dataManager                                  Loading other dataset: CaloChallenge2
[19:40:49.179] INFO   data.dataManager                                  Keys: ['incident_energies', 'showers']
[19:40:54.336] INFO   data.dataManager                                  dict_keys(['incident_energies', 'showers'])
[19:40:54.353] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7fd06ef59040>: 79999 events, 79 batches
[19:40:54.354] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7fd06edb4590>: 10001 events, 10 batches
[19:40:54.354] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7fd115141a00>: 9999 events, 10 batches
[19:40:54.356] INFO   model.modelCreator                                ::Creating Model
[19:40:55.483] INFO   dwave.cloud.client.base                           Fetching de

cuda:1
encoder._networks.0.seq1.0.conv.weight True
encoder._networks.0.seq1.0.conv.bias True
encoder._networks.0.seq1.1.weight True
encoder._networks.0.seq1.1.bias True
encoder._networks.0.seq1.2.weight True
encoder._networks.0.seq1.3.conv.weight True
encoder._networks.0.seq1.3.conv.bias True
encoder._networks.0.seq1.4.weight True
encoder._networks.0.seq1.4.bias True
encoder._networks.0.seq1.5.weight True
encoder._networks.0.seq1.6.conv.weight True
encoder._networks.0.seq1.6.conv.bias True
encoder._networks.0.seq1.7.weight True
encoder._networks.0.seq1.7.bias True
encoder._networks.0.seq1.8.weight True
encoder._networks.0.seq2.0.conv.weight True
encoder._networks.0.seq2.0.conv.bias True
encoder._networks.0.seq2.1.weight True
encoder._networks.0.seq2.1.bias True
encoder._networks.0.seq2.2.weight True
encoder._networks.0.seq2.3.conv.weight True
encoder._networks.0.seq2.3.conv.bias True
encoder._networks.0.seq2.4.weight True
encoder._networks.1.seq1.0.conv.weight True
encoder._networks.1.

In [31]:
import torch.nn as nn
d = nn.Conv3d(2, 4, 1)

In [52]:
[d.weight[i,:,:,:].sum() + d.bias[i] for i in range(4)]
# d.bias

[tensor(0.5458, grad_fn=<AddBackward0>),
 tensor(0.6214, grad_fn=<AddBackward0>),
 tensor(-0.5887, grad_fn=<AddBackward0>),
 tensor(-0.9896, grad_fn=<AddBackward0>)]

In [40]:
d(torch.ones(1,2,1, 1, 1)).flatten()

tensor([ 0.5458,  0.6214, -0.5887, -0.9896], grad_fn=<ViewBackward0>)